# Item-based Models for Movielens

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

We import the necessary libraries for the modelling:

- Pandas: Data manipulation and analysis library in Python, primarily for structured data using Series and DataFrame.

- NumPy: Fundamental library for numerical computing with support for large arrays and mathematical functions.

- os: Python module for interacting with the operating system, used for file and directory operations.

In [2]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'final_strat_sample_parquet',
 'movielens_parquet',
 'movielens_parquet_small',
 'netflix_parquet',
 'sample_tenth_netflix',
 'strat_sample_movielens',
 'strat_sample_netflix']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/final_strat_sample_parquet')

In [6]:
# We print shape of the dataframe
df.shape

(31384, 25)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,Action,Adventure,Drama,Romance,...,Horror,Mystery,Crime,Fantasy,Sci-Fi,War,Film-Noir,Documentary,Western,(no genres listed)
0,82447,"Tomorrow, When the War Began",2010,88838,4.0,2021-09-18,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,82447,"Tomorrow, When the War Began",2010,27561,2.0,2011-01-01,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,82447,"Tomorrow, When the War Began",2010,100696,3.0,2021-12-29,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,82447,"Tomorrow, When the War Began",2010,115974,4.5,2015-10-23,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,82447,"Tomorrow, When the War Began",2010,203230,3.5,2011-01-25,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'Action',
       'Adventure', 'Drama', 'Romance', 'Thriller', 'Comedy', 'Animation',
       'Children', 'Musical', 'Horror', 'Mystery', 'Crime', 'Fantasy',
       'Sci-Fi', 'War', 'Film-Noir', 'Documentary', 'Western',
       '(no genres listed)'],
      dtype='object')

In [9]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


25123
400


In [10]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
215490    136
189614     37
48766      22
10730      22
183281     17
         ... 
213876      1
269025      1
140610      1
303249      1
225484      1
Name: count, Length: 25123, dtype: int64
movieId
8874      3218
161       3087
104       2798
81591     2509
1358      1787
          ... 
146190       1
150266       1
253236       1
202641       1
206323       1
Name: count, Length: 400, dtype: int64


In [11]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

We check the number of different movies that we have in the dataset and the number of unique users we have.

In [12]:
# Print the number of unique users
print("Number of unique users:", df['userId'].nunique())

# Print the number of unique movies
print("Number of unique movies:", df['movieId'].nunique())


Number of unique users: 25123
Number of unique movies: 400


In [13]:
# print the data types of the DF
df.dtypes

movieId                 int64
title                  object
year                   object
userId                  int64
rating                float64
date                   object
Action                  int64
Adventure               int64
Drama                   int64
Romance                 int64
Thriller                int64
Comedy                  int64
Animation               int64
Children                int64
Musical                 int64
Horror                  int64
Mystery                 int64
Crime                   int64
Fantasy                 int64
Sci-Fi                  int64
War                     int64
Film-Noir               int64
Documentary             int64
Western                 int64
(no genres listed)      int64
dtype: object

In [14]:
# We select the columns that are not genres
columns_not_genres = ['movieId', 'title', 'year', 'userId', 'rating', 'date', '(no genres listed)']
for column in df.columns:
    if column not in columns_not_genres :
        df.rename(columns={column: 'genre_' + column}, inplace=True)


In [15]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'genre_Action',
       'genre_Adventure', 'genre_Drama', 'genre_Romance', 'genre_Thriller',
       'genre_Comedy', 'genre_Animation', 'genre_Children', 'genre_Musical',
       'genre_Horror', 'genre_Mystery', 'genre_Crime', 'genre_Fantasy',
       'genre_Sci-Fi', 'genre_War', 'genre_Film-Noir', 'genre_Documentary',
       'genre_Western', '(no genres listed)'],
      dtype='object')

Now we get rid of the cold start problem, we are not gonna give recommendation to the users if they haven't reviewed at least 5 movies.

In [16]:
# Group the DataFrame by 'userId' and count the number of ratings per user
user_rating_counts = df.groupby('userId').size()

# Filter out users who have rated less than 3 movies
active_users = user_rating_counts[user_rating_counts >= 3].index.tolist()

# Select rows from the original DataFrame for active users
df = df[df['userId'].isin(active_users)]

# Display the filtered DataFrame
df.head()

,movieId,title,year,userId,rating,date,genre_Action,genre_Adventure,genre_Drama,genre_Romance,...,genre_Horror,genre_Mystery,genre_Crime,genre_Fantasy,genre_Sci-Fi,genre_War,genre_Film-Noir,genre_Documentary,genre_Western,(no genres listed)
4,82447,"Tomorrow, When the War Began",2010,203230,3.5,2011-01-25,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,82447,"Tomorrow, When the War Began",2010,247373,3.5,2019-02-14,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
11,82447,"Tomorrow, When the War Began",2010,57351,4.0,2011-03-10,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
13,82447,"Tomorrow, When the War Began",2010,60099,3.5,2011-07-04,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
14,82447,"Tomorrow, When the War Began",2010,126357,2.5,2014-06-20,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [17]:
import pandas as pd
import numpy as np

# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Determine the number of unique users and movies
num_users = len(all_user_ids)
num_movies = len(all_movie_ids)

# Create a dictionary to store ratings for each user
user_ratings = {}

# Iterate through the DataFrame and populate the user_ratings dictionary
for _, row in df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    
    # Initialize the user's ratings list if not already present
    if user_id not in user_ratings:
        user_ratings[user_id] = np.zeros(num_movies)
    
    # Assign the rating to the corresponding movie index
    user_ratings[user_id][np.where(all_movie_ids == movie_id)[0][0]] = rating

# Create user-item matrix from the dictionary
item_user_matrix = pd.DataFrame(user_ratings.values(), index=user_ratings.keys(), columns=all_movie_ids)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [18]:
item_user_matrix.head()

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
203230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57351,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60099,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
item_user_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [20]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (704, 323)
Validation set shape: (177, 323)
Test set shape: (221, 323)
 
Training set matrix shape: (704, 323)
Validation set matrix shape: (177, 323)
Test set matrix shape: (221, 323)


In [21]:
train_matrix

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
95283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0
146392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20783,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
330221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252352,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182628,0.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287790,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.]])

In [23]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




### Similarity Calculation utilizing Genres:



In [24]:
import numpy as np

# We create item-genre matrix
def create_item_genre_matrix(df, train):
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Extract item IDs
    item_ids = train.columns
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = np.zeros((len(genre_columns), len(item_ids)))
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        item_id = row['movieId']  # Assuming 'movieId' is the column containing item IDs
        if item_id in item_ids:
            item_index = np.where(item_ids == item_id)[0][0]
            genres = row[genre_columns].values
            item_genre_matrix[:, item_index] = genres
    
    return item_genre_matrix

# We create the item_genre matrix with the train set and the original df
item_genre_matrix = create_item_genre_matrix(df, train_matrix)


In [25]:
item_genre_matrix

item_genre_matrix.shape

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(18, 323)

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [26]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_matrix(data):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(data.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# We calculate the item similarity using the train user-item matrix data
item_similarity_matrix_train = calculate_item_similarity_matrix(train)
print(item_similarity_matrix_train)


[[1.         0.01307211 0.09843998 ... 0.         0.         0.        ]
 [0.01307211 1.         0.1717849  ... 0.         0.         0.        ]
 [0.09843998 0.1717849  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def calculate_item_similarity_genres(item_genre_matrix):
    # Convert the item-genre matrix to a sparse representation
    sparse_item_genre_matrix = sparse.csr_matrix(item_genre_matrix)
    
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(sparse_item_genre_matrix.T)
    
    # Set diagonal values to 1
    np.fill_diagonal(item_similarity_matrix, 1)
    
    return item_similarity_matrix

# Example usage:
# Assuming item_genre_matrix is the item-genre matrix
item_similarity_genres_matrix = calculate_item_similarity_genres(item_genre_matrix)
print(item_similarity_genres_matrix)


[[1.         0.         0.57735027 ... 0.         0.57735027 1.        ]
 [0.         1.         0.         ... 0.5        0.         0.        ]
 [0.57735027 0.         1.         ... 0.         0.33333333 0.57735027]
 ...
 [0.         0.5        0.         ... 1.         0.         0.        ]
 [0.57735027 0.         0.33333333 ... 0.         1.         0.57735027]
 [1.         0.         0.57735027 ... 0.         0.57735027 1.        ]]


In [28]:
item_similarity_genres_matrix.shape

(323, 323)

In [29]:
movie_id_to_index

{43: 0,
 104: 1,
 161: 2,
 222: 3,
 309: 4,
 325: 5,
 326: 6,
 400: 7,
 781: 8,
 791: 9,
 828: 10,
 900: 11,
 1024: 12,
 1179: 13,
 1358: 14,
 1679: 15,
 1711: 16,
 2005: 17,
 2483: 18,
 2763: 19,
 2767: 20,
 3002: 21,
 3094: 22,
 3188: 23,
 3301: 24,
 3314: 25,
 3375: 26,
 3413: 27,
 3627: 28,
 3664: 29,
 3748: 30,
 3834: 31,
 3840: 32,
 4104: 33,
 4155: 34,
 4188: 35,
 4210: 36,
 4359: 37,
 4377: 38,
 4402: 39,
 4452: 40,
 4480: 41,
 4525: 42,
 4607: 43,
 4650: 44,
 4914: 45,
 5014: 46,
 5268: 47,
 5516: 48,
 5733: 49,
 5907: 50,
 6443: 51,
 6557: 52,
 6595: 53,
 6602: 54,
 6738: 55,
 6912: 56,
 7025: 57,
 7122: 58,
 7212: 59,
 7245: 60,
 7358: 61,
 7390: 62,
 7411: 63,
 7460: 64,
 7719: 65,
 8403: 66,
 8426: 67,
 8447: 68,
 8530: 69,
 8774: 70,
 8827: 71,
 8874: 72,
 25871: 73,
 26536: 74,
 26569: 75,
 26682: 76,
 27616: 77,
 27644: 78,
 27679: 79,
 30747: 80,
 30896: 81,
 31613: 82,
 32655: 83,
 33152: 84,
 33912: 85,
 37277: 86,
 40738: 87,
 41353: 88,
 46948: 89,
 47146: 90,
 471

In [30]:
print(index_to_user_id)

{0: 203230, 1: 247373, 2: 57351, 3: 60099, 4: 126357, 5: 229040, 6: 33457, 7: 226138, 8: 240152, 9: 83880, 10: 174815, 11: 94525, 12: 137062, 13: 321330, 14: 165858, 15: 158809, 16: 95264, 17: 38875, 18: 179074, 19: 285399, 20: 154145, 21: 83228, 22: 249747, 23: 290818, 24: 216132, 25: 291126, 26: 26416, 27: 151673, 28: 33012, 29: 96968, 30: 169788, 31: 200441, 32: 26014, 33: 141430, 34: 17963, 35: 9068, 36: 97062, 37: 242363, 38: 89017, 39: 104282, 40: 150068, 41: 215913, 42: 22989, 43: 155990, 44: 156296, 45: 78448, 46: 330535, 47: 52626, 48: 262716, 49: 223842, 50: 12556, 51: 154641, 52: 80177, 53: 133836, 54: 97841, 55: 150310, 56: 64138, 57: 214436, 58: 230807, 59: 88854, 60: 113802, 61: 158661, 62: 69241, 63: 152052, 64: 78134, 65: 236851, 66: 177354, 67: 278686, 68: 185819, 69: 131962, 70: 318402, 71: 87290, 72: 286438, 73: 247956, 74: 258294, 75: 324022, 76: 43710, 77: 167579, 78: 183418, 79: 131184, 80: 201707, 81: 218366, 82: 147899, 83: 47182, 84: 39505, 85: 160061, 86: 1530

In [31]:
print(user_id_to_index)

{203230: 0, 247373: 1, 57351: 2, 60099: 3, 126357: 4, 229040: 5, 33457: 6, 226138: 7, 240152: 8, 83880: 9, 174815: 10, 94525: 11, 137062: 12, 321330: 13, 165858: 14, 158809: 15, 95264: 16, 38875: 17, 179074: 18, 285399: 19, 154145: 20, 83228: 21, 249747: 22, 290818: 23, 216132: 24, 291126: 25, 26416: 26, 151673: 27, 33012: 28, 96968: 29, 169788: 30, 200441: 31, 26014: 32, 141430: 33, 17963: 34, 9068: 35, 97062: 36, 242363: 37, 89017: 38, 104282: 39, 150068: 40, 215913: 41, 22989: 42, 155990: 43, 156296: 44, 78448: 45, 330535: 46, 52626: 47, 262716: 48, 223842: 49, 12556: 50, 154641: 51, 80177: 52, 133836: 53, 97841: 54, 150310: 55, 64138: 56, 214436: 57, 230807: 58, 88854: 59, 113802: 60, 158661: 61, 69241: 62, 152052: 63, 78134: 64, 236851: 65, 177354: 66, 278686: 67, 185819: 68, 131962: 69, 318402: 70, 87290: 71, 286438: 72, 247956: 73, 258294: 74, 324022: 75, 43710: 76, 167579: 77, 183418: 78, 131184: 79, 201707: 80, 218366: 81, 147899: 82, 47182: 83, 39505: 84, 160061: 85, 153067: 

In [32]:
item_user_array.shape

(1102, 323)

In [33]:
item_user_matrix.shape

(1102, 323)

In [34]:
item_similarity_matrix_train.shape

(323, 323)

### Matrices Summation:

Once we have both similarity matrices (the one from the ratings and the one from the genres) we can add them with a weighted sumation.

In [35]:
import numpy as np

def combine_similarity_matrices(similarity_matrix1, similarity_matrix2, weight1=0.7, weight2=0.3):
    # Check if the matrices have the same shape
    if similarity_matrix1.shape != similarity_matrix2.shape:
        raise ValueError("Matrices must have the same shape.")
    
    # Combine matrices using weighted summation directly without creating an intermediate array
    np.multiply(similarity_matrix1, weight1, out=similarity_matrix1)
    np.multiply(similarity_matrix2, weight2, out=similarity_matrix2)
    np.add(similarity_matrix1, similarity_matrix2, out=similarity_matrix1)
    return similarity_matrix1

# Define weights for each matrix
weight_ratings = 0.7
weight_genres = 0.3

# Combine the similarity matrices
item_similarity_matrix_train = combine_similarity_matrices(item_similarity_matrix_train, item_similarity_genres_matrix, weight_ratings, weight_genres)

# Print or use the combined similarity matrix as needed
print("Combined Similarity Matrix:\n", item_similarity_matrix_train)


Combined Similarity Matrix:
 [[1.         0.00915048 0.24211307 ... 0.         0.17320508 0.3       ]
 [0.00915048 1.         0.12024943 ... 0.15       0.         0.        ]
 [0.24211307 0.12024943 1.         ... 0.         0.1        0.17320508]
 ...
 [0.         0.15       0.         ... 1.         0.         0.        ]
 [0.17320508 0.         0.1        ... 0.         1.         0.17320508]
 [0.3        0.         0.17320508 ... 0.         0.17320508 1.        ]]


### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in the dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [36]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{43: [781, 3314, 1358, 5014, 173417], 104: [132888, 4104, 1679, 221792, 4452], 161: [81591, 7025, 1358, 99145, 265432], 222: [4155, 31613, 3094, 40738, 4914], 309: [286843, 90116, 188509, 185921, 185421], 325: [1679, 172365, 210183, 70762, 101379], 326: [201384, 400, 54519, 170545, 185921], 400: [201384, 54519, 153056, 140946, 27644], 781: [43, 185421, 1358, 326, 5014], 791: [113432, 113790, 100743, 186279, 182845], 828: [6557, 132130, 164859, 158813, 128850], 900: [6912, 222, 3094, 40738, 3834], 1024: [128679, 164859, 196243, 4188, 186075], 1179: [1711, 1358, 90116, 90108, 222], 1358: [185421, 5014, 43, 326, 781], 1679: [325, 104, 3664, 4104, 32655], 1711: [1179, 173417, 105028, 74727, 1358], 2005: [158813, 104, 98697, 112897, 82447], 2483: [237308, 278804, 280174, 81656, 173731], 2763: [92393, 175569, 144274, 6595, 2005], 2767: [195451, 153552, 210183, 101379, 162228], 3002: [113790, 153258, 78459, 113432, 169434], 3094: [33912, 3413, 40738, 8530, 49115], 3188: [49115, 791, 204838, 2

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [37]:
# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
95283,3.833333,3.833333,5.0,3.833333,3.833333,3.833333,3.833333,3.833333,3.833333,3.833333,...,3.833333,5.0,3.833333,5.0,3.833333,3.833333,3.833333,3.833333,3.5,3.833333
146392,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
155015,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,...,2.428571,2.428571,2.428571,2.5,2.428571,2.428571,2.428571,2.428571,3.0,2.428571
20783,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
330221,3.5,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.5,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667


In [38]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [39]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [40]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Crimson Tide ',
  'Betrayal ',
  'The Unforgivable ',
  'Read My Lips (Sur mes lèvres) ',
  'The Last Shift '],
 ['Malamaal Weekly ',
  'Restoration ',
  'The War of the Worlds: Next Century ',
  'Happy Gilmore ',
  'Salt for Svanetia '],
 ['Black Belly of the Tarantula ',
  'The Other Hell ',
  'Midnight in Paris ',
  'Exit Humanity ',
  "Satan's Slaves 2: Communion "],
 ['Felix and Meira ',
  'Days of Eclipse ',
  'The War of the Worlds: Next Century ',
  'Black Swan ',
  'Salt for Svanetia '],
 ['Brain Damage ',
  'Midnight in Paris ',
  'Munchies ',
  'Comedy of Terrors, The ',
  'My Son '],
 ['Shaun of the Dead ',
  'Sarah Plays a Werewolf ',
  'Downtown ',
  'Seven Year Itch, The ',
  'Comedy Central Roast of James Franco '],
 ['Restoration ',
  'If There Be Thorns ',
  'Munchies ',
  'Comedy of Terrors, The ',
  'Stealing Beauty '],
 ['Read My Lips (Sur mes lèvres) ',
  'Betrayal ',
  "Don't Worry Darling ",
  'WarGames: The Dead Code ',
  'Crimson Tide '],
 ['Five Element Ni

704

### Step 5: Model Evaluation
- We evaluate the performance of the ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)
- Split the dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [41]:
train.shape

predicted_ratings_train.shape

(704, 323)

(704, 323)

In [42]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# Assuming train is the numpy array or DataFrame representing the item-user matrix
# Assuming predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.662820862755312
Root Mean Squared Error (RMSE) on Training Data: 0.9044795743176157


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of the ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows us to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [44]:
# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [45]:
predicted_ratings_val.head()

predicted_ratings_val.shape

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
123628,4.5,4.166667,4.5,4.166667,4.166667,4.166667,4.166667,4.166667,4.5,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
9448,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125,...,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125,3.125
193244,2.333333,2.0,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,...,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333
185341,3.666667,3.666667,3.666667,4.0,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667
324596,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625,...,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625,2.625


(177, 323)

Now we evaluate the model with the Validation item-user matrix.

In [46]:
# Now, we can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.6573045517018352
Root Mean Squared Error (RMSE) on Validation Data: 0.8928317780103675


### Retraining of the model with the Train-Val set

In [47]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We combine the similarity matrices (the retrained one and the genres)
item_similarity_matrix_train_val = combine_similarity_matrices(item_similarity_matrix_train_val, item_similarity_genres_matrix)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [48]:
test_matrix

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
80664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71676,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170229,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160959,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176710,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [50]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
    
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, we can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set: 0.6657759733106727
Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set: 0.9245576937034555


In [51]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['The Frontier ',
  'Housemaid, The (Hanyo) ',
  'Sombre ',
  'Russian Doll ',
  "Money Money Money (L'aventure, c'est l'aventure) "],
 ['Lorelei ',
  'Black Swan ',
  'Women, The ',
  "Duchess of Langeais, The (a.k.a. Don't Touch the Axe) (Ne touchez pas la hache) ",
  'Songwriter '],
 ['The Frontier ',
  'The Midwife ',
  'Housemaid, The (Hanyo) ',
  'Russian Doll ',
  'Death of Mr. Lazarescu, The (Moartea domnului Lazarescu) '],
 ['Midnight in Paris ',
  'Black Swan ',
  '211 ',
  'Son of Lassie ',
  "Satan's Slaves 2: Communion "],
 ['The Executioners ',
  'Goonies, The ',
  'Happy Gilmore ',
  'Son of Lassie ',
  "Satan's Slaves 2: Communion "],
 ['Uncle Yanco ', 'The Executioners ', 'Saw ', 'My Son ', 'Son of Lassie '],
 ['Sweet November ',
  'Felix and Meira ',
  'Restoration ',
  'Sarah Plays a Werewolf ',
  'Women, The '],
 ['Black Belly of the Tarantula ',
  'The War of the Worlds: Next Century ',
  'The Unforgivable ',
  'Impossible, The (Imposible, Lo) ',
  'The Last Shift

Size of top_recommendations_test: 221


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- We have to create a dataset with a user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Additionally we will create a item-genre matrix to compute the similarity between objects using the genres.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [52]:
item_similarity_matrix_train.shape

(323, 323)

In [53]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# We have an item-item similarity matrix 'item_similarity_matrix_train'
# and we want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [54]:
len(item_neighborhoods_classification_train)

323

### Step 3: Ratings classification selection

In [55]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings


# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


          43        104       161       222       309       325       326     \
95283   3.833333  3.833333  5.000000  3.833333  3.833333  3.833333  3.833333   
146392  3.000000  3.000000  3.000000  3.000000  3.000000  3.000000  3.000000   
155015  2.428571  2.428571  2.428571  2.428571  2.428571  2.428571  2.428571   
20783   4.166667  4.166667  3.377383  4.166667  4.166667  4.166667  4.166667   
330221  3.500000  3.666667  3.666667  3.666667  3.666667  3.666667  3.666667   

          400       781       791     ...    263415    265432    265548  \
95283   3.833333  3.833333  3.833333  ...  3.833333  5.000000  3.833333   
146392  3.000000  3.000000  3.000000  ...  3.000000  3.000000  3.000000   
155015  2.428571  2.428571  2.428571  ...  2.428571  2.428571  2.428571   
20783   4.166667  4.166667  4.166667  ...  4.166667  4.166667  4.166667   
330221  3.666667  3.666667  3.666667  ...  3.666667  3.666667  3.666667   

          276897    277764    278478    278804    280174    281512  

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9732\1007744238.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)


In [56]:
type(train_matrix)

pandas.core.frame.DataFrame

In [57]:
predicted_ratings_classification_train

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
95283,3.833333,3.833333,5.000000,3.833333,3.833333,3.833333,3.833333,3.833333,3.833333,3.833333,...,3.833333,5.000000,3.833333,3.377383,3.833333,3.833333,3.833333,3.833333,3.500000,3.833333
146392,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
155015,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,...,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,2.428571,3.000000,2.428571
20783,4.166667,4.166667,3.377383,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
330221,3.500000,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252352,4.500000,3.500000,3.500000,3.500000,3.500000,3.377383,3.500000,3.500000,3.377383,3.500000,...,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
182628,2.000000,2.000000,3.377383,3.377383,2.000000,2.000000,3.377383,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
325578,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,...,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000,3.250000
287790,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.377383,3.800000,4.000000,3.800000,...,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.000000,3.800000


### Step 4: Recommendations Generation

In [58]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 95283 recommendations: ['Impossible, The (Imposible, Lo) ', 'The Unforgivable ', 'WarGames: The Dead Code ', 'Crimson Tide ', 'The Last Shift ']
User 146392 recommendations: ['Malamaal Weekly ', 'Restoration ', 'The War of the Worlds: Next Century ', 'Happy Gilmore ', 'Salt for Svanetia ']
User 155015 recommendations: ['Blob, The ', 'Exit Humanity ', 'Fate ', 'Post Tenebras Lux ', "Satan's Slaves 2: Communion "]
User 20783 recommendations: ['Felix and Meira ', 'Days of Eclipse ', 'The War of the Worlds: Next Century ', 'Black Swan ', 'Salt for Svanetia ']
User 330221 recommendations: ['Midnight in Paris ', 'The President ', 'Munchies ', '211 ', 'My Son ']
User 316107 recommendations: ['Goonies, The ', 'Secret Zoo ', 'Debt Collectors ', 'Gentlemen of Fortune (Dzhentlmeny udachi) ', 'Chairman of the Board ']
User 163257 recommendations: ['Restoration ', 'Midnight in Paris ', 'Pain and Glory ', 'Munchies ', 'Stealing Beauty ']
User 269463 recommendations: ['Impossible, The (Imposible

### Step 5: Model Evaluation (Item KNN Classification)

In [59]:
import numpy as np
import pandas as pd

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# Assuming train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8085203046476831
Root Mean Squared Error (RMSE) on Training Data: 1.0320396359211221


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of the ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [60]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [61]:
# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue

            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_9732\3630870256.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [62]:
recalculated_val_predictions

,43,104,161,222,309,325,326,400,781,791,...,263415,265432,265548,276897,277764,278478,278804,280174,281512,286843
123628,4.500000,3.397480,4.500000,4.166667,4.166667,4.166667,4.166667,4.166667,4.500000,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
9448,3.125000,3.125000,3.397480,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,...,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000,3.125000
193244,2.333333,3.397480,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,...,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333
185341,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667
324596,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,...,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239153,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
153297,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
204888,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
230023,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,...,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667,2.666667


In [63]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.8669585372439459
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.0734580076910518


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [64]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9732\3630870256.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [65]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.8285825326733091
Root Mean Squared Error (RMSE) on Test Data: 1.0646684554238792
